# Lab 10 : Neural Networks


# 1. Load the dataset (X and Y vector)

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
import pandas as pd
import seaborn as sns 

DATASET_PATH = '../data/Fish_Dataset/Fish_Dataset/'

In [27]:
def get_labels():
    folders = os.listdir(DATASET_PATH)
    labels = []
    for f in folders:
        if not('.m' in f) and not('.txt' in f):
            labels.append(f)
    labels = np.array(labels)
    return labels

def get_X_Y(labels):
    # Get the images for each label
    img_shape = (128, 128) #resizing images to this shape ( after converting color image to gray image)
    n_sample_per_class = 600 #Keep only this number of samples per class (expensive complexity issue)
    nb_img = n_sample_per_class*len(labels) #1000 images per class

    X = np.zeros((nb_img, img_shape[0]*img_shape[1])) # feature matrix
    Y = np.zeros(nb_img) #labels vector
    for i in range(len(labels)):
        folder = os.path.join(DATASET_PATH, labels[i], labels[i])
        images = os.listdir(folder)
        np.random.shuffle(images)
        for j in range(n_sample_per_class):
            img_path = os.path.join(folder, images[j])
            img = cv2.imread(img_path, 0) #Load the gray image
            #Resizing
            img = cv2.resize(src=img, dsize=img_shape)
            #Normalization
            img = img/255
            #Add the image in the feature matrix and its labels to the label vector
            X[i*n_sample_per_class+j, :] = img.flatten()
            Y[i*n_sample_per_class+j] = np.argwhere(labels==labels[i])[0]
    return X, Y

labels = get_labels()
X, Y = get_X_Y(labels)
#print("Feature matrix : \n", X)
#print("Label vector : \n", Y)

pca = PCA(n_components=50)
pca.fit(X)
X = pca.transform(X)

# 80% train set
X_train, X_rem, y_train, y_rem = train_test_split(X, Y, train_size=0.8, random_state=42)

# 10% validation, 10% test
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5, random_state=42)

# Shape will inform neural network architecture.
print(X_train.shape)

(4320, 50)


# 2. Keras Classifier

# 2.1 Keras Initialisation

In [58]:
import tensorflow as tf
from tensorflow import keras

# Set up session
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [59]:
# Define model - no 'Flatten' needed as PCA flattens images already.
neural_classifier = keras.models.Sequential([
    keras.layers.Dense(50),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(9, activation="softmax")
    ])

# Compile model
neural_classifier.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
    

# Fit to data
history = neural_classifier.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid))


Epoch 1/30
135/135 [==============================] - 1s 2ms/step - loss: 1.4133 - accuracy: 0.5065 - val_loss: 1.0292 - val_accuracy: 0.6519
Epoch 2/30
135/135 [==============================] - 0s 1ms/step - loss: 0.8542 - accuracy: 0.7021 - val_loss: 0.8788 - val_accuracy: 0.6796
Epoch 3/30
135/135 [==============================] - 0s 2ms/step - loss: 0.6192 - accuracy: 0.7949 - val_loss: 0.7259 - val_accuracy: 0.7352
Epoch 4/30
135/135 [==============================] - 0s 1ms/step - loss: 0.4773 - accuracy: 0.8438 - val_loss: 0.5962 - val_accuracy: 0.7889
Epoch 5/30
135/135 [==============================] - 0s 1ms/step - loss: 0.3695 - accuracy: 0.8769 - val_loss: 0.5939 - val_accuracy: 0.8037
Epoch 6/30
135/135 [==============================] - 0s 1ms/step - loss: 0.3031 - accuracy: 0.9037 - val_loss: 0.5153 - val_accuracy: 0.8259
Epoch 7/30
135/135 [==============================] - 0s 1ms/step - loss: 0.2338 - accuracy: 0.9238 - val_loss: 0.5720 - val_accuracy: 0.8167
Epoch 

# 2.2 Pipeline Conversion

In [60]:
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier

a_classifier = keras.models.Sequential([
    keras.layers.Dense(50),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(9, activation="softmax")
    ])

a_classifier.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

a_pipe = Pipeline(steps=[('nn', KerasClassifier(neural_classifier, verbose=0))])

a_params_grid = {
    'nn__batch_size' : [10, 20, 40],
    'nn__epochs' : [10, 50, 100]
}

b_classifier = keras.models.Sequential([
    keras.layers.Dense(50),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(200, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(9, activation="softmax")
    ])

b_classifier.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

b_pipe = Pipeline(steps=[('nn', KerasClassifier(neural_classifier, verbose=0))])

b_params_grid = {
    'nn__batch_size' : [10, 20, 40],
    'nn__epochs' : [10, 50, 100]
}



In [61]:
def run_grid_search(pipeline, params):
    search = GridSearchCV(estimator=pipeline, param_grid=params, scoring='accuracy', cv=10)
    search.fit(X_train, y_train)
    search.score(X_train, y_train)
    print("Best params found : ", search.best_params_)
    print("Score : ", round(search.score(X_valid, y_valid), 3)) 
    return search

a_search = run_grid_search(a_pipe, a_params_grid)
b_search = run_grid_search(b_pipe, b_params_grid)

INFO:tensorflow:Assets written to: ram:///tmp/tmpwygdp7z5/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmpxotn3rtg/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmp5ecpol73/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmpst_ovt1d/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmpq0k6lvmz/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmptgmftbic/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmph0bc79vj/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmp39j17zv0/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmptpv1yqx2/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmp4jd9csbd/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmp0gkh9917/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmpcnk4bloc/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmprvu00wy0/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmpad8cmtph/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmplxl8dkgn/assets
INFO:tensorflow:Assets wr

INFO:tensorflow:Assets written to: ram:///tmp/tmp3814i42r/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmpmk_hl4g1/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmptelab97u/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmpbrfm10z3/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmpd0lzwln8/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmp0nzi9tel/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmp8dl8yjbz/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmpmarmmq0t/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmp1v37dhze/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmptt98t6vs/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmp_g__b34l/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmp831r3tyy/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmpe95wg99k/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmpe2jw80s2/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmpbxi772xi/assets
INFO:tensorflow:Assets wr

# 3. Evaluation